Aakash

EP21BTECH11001

HW-4

In [10]:
import numpy as np

1 Numpy Implementation
In this assignment you will implement each of the components of a Convolutional Neural Network (CNN)

from scratch (i.e., without using built-in functions for convolution, pooling, non-linearity, padding, strid-
ing). Your implementation must accept an image input and generate an output vector. Use random weights

for filter kernels and fully connected layers. Specifically, implement the following:

1. Convolution function: It accepts as input an image, a filter kernel, stride, padding and the non-linear
function. The function must correlate (convolve) the input image (after padding if specified) with
the kernel (at the specified stride size) and generate an output activation after applying the specified
non-linearity. Verify with the standard options for the non-linear activation functions - sigmoid,
tanh, ReLU, Parametric ReLU (PReLU). Display the input image, the filter kernel and the output
activation map. Ensure that your function can accept multi-channel input and a corresponding kernel
volume. (3)

In [11]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def tanhx(x):
    return np.tanh(x)

def relu(x):
    return np.maximum(0,x)

def prelu(x,alpha=0.1):
    return np.where(x>0,x,alpha*x)


In [ ]:
def avgPooling(input,output,kernel_h,kernel_w,stride):
    input_h,input_w,channels=input.shape
    output_h,output_w,channels=output.shape
    for c in range(channels):
        for h in range(output_h):
            for w in range(output_w):
                sum=0.0
                for i in range(kernel_h):
                    for j in range(kernel_w):
                        sum+=input[(h*stride)+i][(w*stride)+j][c]
                sum/=(kernel_h*kernel_w)
                output[h*stride][w*stride]=sum
    return output



def maxPooling(input,output,kernel_h,kernel_w,stride):
    input_h,input_w,channels=input.shape
    output_h,output_w,channels=output.shape
    for c in range(channels):
        for h in range(output_h):
            for w in range(output_w):
                max = float('-inf')
                for i in range(kernel_h):
                    for j in range(kernel_w):
                        res=max(input[(h*stride)+i][(w*stride)+j][c],res)
                output[h*stride][w*stride]=res
    return output


def minPooling(input,output,kernel_h,kernel_w,stride):
    input_h,input_w,channels=input.shape
    output_h,output_w,channels=output.shape
    for c in range(channels):
        for h in range(output_h):
            for w in range(output_w):
                res = float('inf')
                for i in range(kernel_h):
                    for j in range(kernel_w):
                        res=min(input[(h*stride)+i][(w*stride)+j][c],res)
                output[h*stride][w*stride]=res
    return output

In [12]:
#1. first of all implement convolution function
# input: image filter/kernel/stride/padding/non-linear fctn
# conventions: image_h/image_w/channels/kernel_h/kernel_w/kernel_channels
# checkconditns: channels==kernel_channels/

def Convolution(image,kernel,stride,padding,activation):

    #add padding first to image with filling 0 to padding cells
    image = np.pad(image, ((padding, padding), (padding, padding), (0, 0)), mode='constant')

    # size of imputs
    image_h,image_w,channels=image.shape
    kernel_h,kernel_w,kernel_channels=kernel.shape

    # necessary condition check
    if kernel_channels!=channels:
        raise ValueError("Kernel and image must have same no of channels")

    #now getting the output size   
    # dimension of output matrix -> (n-l+2p)/s+1 
    output_h=int(((image_h-kernel_h)/stride)+1)
    output_w=int(((image_w-kernel_w)/stride)+1)
    output=np.zeros((output_h,output_w,channels))
    for c in range(channels):
        for h in range(output_h):
            for w in range(output_w):
                curr=0
                for i in range(kernel_h):
                    for j in range(kernel_w):
                        curr+=kernel[i][j][c]*image[(h*stride)+i][(w*stride)+j][c]
                output[h*stride][w*stride][c]=curr

    if(activation=='sig'):
        output=sigmoid(output)
    if(activation=='tanhx'):
        output=tanhx(output)
    if(activation=='relu'):
        output=relu(output)
    if(activation=='prelu'):
        output=prelu(output)
    print(image)
    print(kernel)
    print(output)
    return output    

2. Pooling function: It accepts as input the activation map output from the convolution function, a
pooling function, and stride. The function must output the appropriately pooled activation map.
Display the input activation map and the pooled output. (2)

In [14]:
# 2. now will will implement pooling function
# input-> inputmap/pooling function/stride/dimensionofkernel
def Pooling(inputmap,pooling_fctn,stride,kernel_h,kernel_w):

    # getting the inputmap dimensions
    inputmap_h,inputmap_w,channels=inputmap.shape

    # getting the outputmap dimensions
    outputmap_h=int(((inputmap_h-kernel_h)/stride)+1)
    outputmap_w=int(((inputmap_w-kernel_w)/stride)+1)
    outputmap=np.zeros((outputmap_h,outputmap_w,channels))
    if(pooling_fctn=='max'):
        outputmap=maxPooling(inputmap,outputmap,kernel_h,kernel_w,stride)
    
    print(inputmap)
    print(outputmap)
    return outputmap

3. Convolution layer function: It accepts as input a volume (image or activation maps), filter kernels,
stride, padding and the non-linear function. The function must convolve the input volume (after
padding if specified) with each of the kernels (at the specified stride size) and generates an output
activation volume after applying the specified non-linearity. Display the input image or activation
maps, the filter kernels and the output activation maps. Verify that the output of this function does
indeed have the expected size (W × H × C) as discussed in class. (1)

In [1]:
# 3 Convolution layer function
def ConvolutionLayer(input_volume, kernels, stride, padding, activation):
    # Add padding to the input volume
    input_volume = np.pad(input_volume, ((padding, padding), (padding, padding), (0, 0)), mode='constant')

    # Get dimensions of the input and kernel
    input_h, input_w, input_channels = input_volume.shape
    kernel_h, kernel_w, kernel_channels, num_kernels = kernels.shape

    # Check if kernel channels match input channels
    if kernel_channels != input_channels:
        raise ValueError("Kernel and input volume must have the same number of channels")

    # Calculate output dimensions
    output_h = int((input_h - kernel_h) / stride + 1)
    output_w = int((input_w - kernel_w) / stride + 1)
    output_volume = np.zeros((output_h, output_w,kernel_channels, num_kernels))

    # Convolve each kernel with the input volume and store in the output volume
    for k in range(num_kernels):
        output_volume[:, :,:, k] = Convolution(input_volume, kernels[:, :, :, k], stride)

    # Apply activation function
    if activation == 'sig':
        output_volume = sigmoid(output_volume)
    elif activation == 'tanhx':
        output_volume = tanhx(output_volume)
    elif activation == 'relu':
        output_volume = relu(output_volume)
    elif activation == 'prelu':
        output_volume = prelu(output_volume)
    else:
        raise ValueError("Unsupported activation function")

    # Display input volume, kernels, and output volume
    print("Input Volume:\n", input_volume)
    print("Kernels:\n", kernels)
    print("Output Activation Volume:\n", output_volume)
    
    return output_volume

4. Pooling layer function: It accepts as input the activation map volume, the pooling function, stride,
and generates a pooled output volume. Display the input and output volumes. (1)

In [ ]:
# 4. now will will implement pooling function
# input-> inputmap/pooling function/stride/dimensionofkernel
def Pooling(inputmap,pooling_fctn,stride,kernel_h,kernel_w):

    # getting the inputmap dimensions
    inputmap_h,inputmap_w,channels,inputmap_num=inputmap.shape

    # getting the outputmap dimensions
    outputmap_h=int(((inputmap_h-kernel_h)/stride)+1)
    outputmap_w=int(((inputmap_w-kernel_w)/stride)+1)
    outputmap=np.zeros((outputmap_h,outputmap_w,channels,inputmap_num))
    if(pooling_fctn=='max'):
        for k in range(inputmap_num):
            outputmap[:,:,:,k]=maxPooling(inputmap,outputmap,kernel_h,kernel_w,stride)
    elif(pooling_fctn=='min'):
        for k in range(inputmap_num):
            outputmap[:,:,:,k]=minPooling(inputmap,outputmap,kernel_h,kernel_w,stride)
    elif(pooling_fctn=='avg'):
        for k in range(inputmap_num):
            outputmap[:,:,:,k]=avgPooling(inputmap,outputmap,kernel_h,kernel_w,stride)
    
    print(inputmap)
    print(outputmap)
    return outputmap

5. Flattening (unraveling) function: It accepts as input the activation map volume output by the pool-
ing layer and generates a vector of a specified size. It is important to note that this function has a
weight matrix associated with it whose size is chosen such that the input and desired output sizes are
matched. (1)

In [ ]:
#5 flatten function
def flatten(input_volume):
    return input_volume.flatten()

def flatten_with_weights(input_volume, weight_matrix):
    # Flatten the input volume
    flattened_vector = flatten(input_volume)

    if flattened_vector.size != weight_matrix.shape[1]:
        raise ValueError("Flattened input size must match weight matrix input size.")

    output_vector = np.dot(weight_matrix, flattened_vector)
    return output_vector
